In [142]:
import pandas as pd
import numpy  as np
import tensorflow as tf
AUTO = tf.data.experimental.AUTOTUNE
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Input

In [143]:
from google.colab import drive ; drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [287]:
df = pd.read_csv("/content/drive/My Drive/Dataset/train.csv")
df.head()

,id_pasien,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_pekerjaan,jenis_tempat_tinggal,rata2_level_glukosa,bmi,merokok,stroke
0,100000,Laki-laki,43,tidak,tidak,sudah,swasta,Pedesaan,96.39,26.1,mantan perokok,0
1,100001,Perempuan,58,tidak,tidak,sudah,swasta,Pedesaan,114.07,21.2,tidak,0
2,100002,Laki-laki,29,tidak,tidak,belum,swasta,Pedesaan,118.96,29.7,tidak,0
3,100003,Laki-laki,54,tidak,tidak,sudah,Wirausaha,Perkotaan,73.01,27.7,tidak,0
4,100004,Laki-laki,78,ya,ya,sudah,Wirausaha,Perkotaan,243.82,36.3,mantan perokok,0


In [288]:
test = pd.read_csv("/content/drive/My Drive/Dataset/data_test.csv")
test.head(1)

,id_pasien,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_pekerjaan,jenis_tempat_tinggal,rata2_level_glukosa,bmi,merokok
0,2000,Laki-laki,24,tidak,tidak,belum,belum bekerja,Perkotaan,83.45,25.3,tidak


In [146]:
df.isna().sum()

id_pasien               0
jenis_kelamin           0
umur                    0
hipertensi              0
penyakit_jantung        0
sudah_menikah           0
jenis_pekerjaan         0
jenis_tempat_tinggal    0
rata2_level_glukosa     0
bmi                     0
merokok                 0
stroke                  0
dtype: int64

In [289]:
df = df[["jenis_kelamin","umur","hipertensi","penyakit_jantung","sudah_menikah","jenis_pekerjaan","jenis_tempat_tinggal","rata2_level_glukosa","bmi","merokok","stroke"]]
df.head(1)

,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_pekerjaan,jenis_tempat_tinggal,rata2_level_glukosa,bmi,merokok,stroke
0,Laki-laki,43,tidak,tidak,sudah,swasta,Pedesaan,96.39,26.1,mantan perokok,0


In [148]:
df["merokok"].unique()

array(['mantan perokok', 'tidak', 'perokok aktif'], dtype=object)

In [290]:
df["hipertensi"] = df["hipertensi"].replace({"tidak":0,
                                             "ya":1})
df["penyakit_jantung"] = df["penyakit_jantung"].replace({"ya":1,"tidak":0})
df["sudah_menikah"] = df["sudah_menikah"].replace({"sudah":1,"belum":0})
#df["jenis_kelamin"] = df["jenis_kelamin"].replace({"Laki-laki":0,"Perempuan":0})
#df["jenis_tempat_tinggal"] = df["jenis_tempat_tinggal"].replace({"Pedesaan":1,"Perkotaan":2})
#df["merokok"] = df["merokok"].replace({"mantan perokok":1,"perokok aktif":2,"tidak":0})
test["hipertensi"] = test["hipertensi"].replace({"tidak":0,
                                             "ya":1})
test["penyakit_jantung"] = test["penyakit_jantung"].replace({"ya":1,"tidak":0})
test["sudah_menikah"] = test["sudah_menikah"].replace({"sudah":1,"belum":0})
#test["jenis_kelamin"] = test["jenis_kelamin"].replace({"Laki-laki":0,"Perempuan":0})
#test["jenis_tempat_tinggal"] = test["jenis_tempat_tinggal"].replace({"Pedesaan":1,"Perkotaan":2})
#test["merokok"] = test["merokok"].replace({"mantan perokok":1,"perokok aktif":2,"tidak":0})



In [291]:
dummies = pd.get_dummies(df[["jenis_pekerjaan"]])
dummiesTest = pd.get_dummies(test[["jenis_pekerjaan"]])

In [292]:
dummies = pd.get_dummies(df[["jenis_kelamin","jenis_pekerjaan","jenis_tempat_tinggal"]])
dummiesTest = pd.get_dummies(test[["jenis_kelamin","jenis_pekerjaan","jenis_tempat_tinggal"]])

In [293]:
df[dummies.columns] = dummies
test[dummiesTest.columns] = dummiesTest
df.drop(["jenis_pekerjaan"],axis=1,inplace=True)
test.drop(["jenis_pekerjaan"],axis=1,inplace=True)

In [294]:
df.head(1)

,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_tempat_tinggal,rata2_level_glukosa,bmi,merokok,stroke,jenis_kelamin_Laki-laki,jenis_kelamin_Perempuan,jenis_pekerjaan_PNS,jenis_pekerjaan_Wirausaha,jenis_pekerjaan_belum bekerja,jenis_pekerjaan_swasta,jenis_tempat_tinggal_Pedesaan,jenis_tempat_tinggal_Perkotaan
0,Laki-laki,43,0,0,1,Pedesaan,96.39,26.1,mantan perokok,0,1,0,0,0,0,1,1,0


In [295]:
dummies = pd.get_dummies(df["merokok"],drop_first=True)
dummiesTest = pd.get_dummies(test["merokok"],drop_first=True)

In [296]:
df[dummies.columns] = dummies
test[dummiesTest.columns] = dummiesTest

In [297]:
df.drop("merokok",inplace=True,axis=1)
test.drop("merokok",inplace=True,axis=1)

In [298]:
df.head(1)

,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_tempat_tinggal,rata2_level_glukosa,bmi,stroke,jenis_kelamin_Laki-laki,jenis_kelamin_Perempuan,jenis_pekerjaan_PNS,jenis_pekerjaan_Wirausaha,jenis_pekerjaan_belum bekerja,jenis_pekerjaan_swasta,jenis_tempat_tinggal_Pedesaan,jenis_tempat_tinggal_Perkotaan,perokok aktif,tidak
0,Laki-laki,43,0,0,1,Pedesaan,96.39,26.1,0,1,0,0,0,0,1,1,0,0,0


In [299]:
from sklearn.preprocessing import MinMaxScaler

StandardOBJ = MinMaxScaler()
StandardFit = StandardOBJ.fit_transform(df[["rata2_level_glukosa","bmi"]])
StandardFitTest = StandardOBJ.fit_transform(test[["rata2_level_glukosa","bmi"]])
StandardFit

array([[0.17530927, 0.1953602 ],
       [0.25021183, 0.13553114],
       [0.27092866, 0.23931624],
       ...,
       [0.02563125, 0.22100122],
       [0.22508897, 0.18315018],
       [0.05965091, 0.22710623]])

In [300]:
df["rata2_level_glukosa"] = StandardFit[:,0]
df["bmi"] = StandardFit[:,1]
test["rata2_level_glukosa"] = StandardFitTest[:,0]
test["bmi"] = StandardFitTest[:,1]

In [302]:
df.drop(["jenis_kelamin","jenis_tempat_tinggal"],axis=1,inplace=True)
test.drop(["jenis_kelamin","jenis_tempat_tinggal"],axis=1,inplace=True)


In [303]:
dfcopy=df.copy()
dfcopy2 = df.copy()

In [304]:
dfcopy = df[df["stroke"] == 1]
dfcopy2 = df[df["stroke"]==0][:390]
dfcopy2.head(1)

,umur,hipertensi,penyakit_jantung,sudah_menikah,rata2_level_glukosa,bmi,stroke,jenis_kelamin_Laki-laki,jenis_kelamin_Perempuan,jenis_pekerjaan_PNS,jenis_pekerjaan_Wirausaha,jenis_pekerjaan_belum bekerja,jenis_pekerjaan_swasta,jenis_tempat_tinggal_Pedesaan,jenis_tempat_tinggal_Perkotaan,perokok aktif,tidak
0,43,0,0,1,0.175309,0.19536,0,1,0,0,0,0,1,1,0,0,0


In [305]:
finaldf = pd.concat([dfcopy,dfcopy2],axis=0)
finaldf.shape

(773, 17)

In [306]:
from tensorflow.keras.utils import to_categorical
X = finaldf.drop("stroke",axis=1)
Y = finaldf["stroke"]
Y = Y.astype("float64")
X.shape, Y.shape

((773, 16), (773,))

In [307]:
X.head()

,umur,hipertensi,penyakit_jantung,sudah_menikah,rata2_level_glukosa,bmi,jenis_kelamin_Laki-laki,jenis_kelamin_Perempuan,jenis_pekerjaan_PNS,jenis_pekerjaan_Wirausaha,jenis_pekerjaan_belum bekerja,jenis_pekerjaan_swasta,jenis_tempat_tinggal_Pedesaan,jenis_tempat_tinggal_Perkotaan,perokok aktif,tidak
27,45,1,0,0,0.786604,0.352869,1,0,0,0,0,1,0,1,1,0
59,78,0,0,1,0.052025,0.177045,0,1,0,0,0,1,0,1,0,1
246,68,1,0,0,0.104982,0.239316,0,1,0,1,0,0,0,1,0,1
297,68,1,0,1,0.125911,0.222222,1,0,0,0,0,1,0,1,1,0
311,38,0,1,1,0.046518,0.158730,0,1,0,0,0,1,0,1,0,0


In [308]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X,Y , test_size=0.05, random_state=5)
x_train.shape, y_train.shape

((734, 16), (734,))

In [271]:
y_train

86       0.0
241      0.0
71       0.0
18777    1.0
3413     1.0
        ... 
6098     1.0
17       0.0
8586     1.0
323      0.0
15581    1.0
Name: stroke, Length: 734, dtype: float64

In [309]:
TrainDataset = (tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(32).prefetch(AUTO))
ValDataset = (tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(32).cache().prefetch(AUTO))
TrainDataset.element_spec

(TensorSpec(shape=(None, 16), dtype=tf.float64, name=None),
 TensorSpec(shape=(None,), dtype=tf.float64, name=None))

In [168]:
ValDataset.element_spec

(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None),
 TensorSpec(shape=(None,), dtype=tf.float64, name=None))

In [312]:

input = Input(shape=(16,),dtype=tf.float64)
layer = Dense(units=64 ,activation=tf.nn.leaky_relu)(input)
layer = Dropout(0.76)(layer)
layer = Dense(units=32,activation=tf.nn.leaky_relu)(layer)
layer = Dropout(0.4)(layer)
layer = Dense(units=32,activation=tf.nn.relu)(layer)
layer = Dense(16,activation=tf.nn.tanh)(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1, activation=tf.nn.sigmoid)(layer)

METRICS = [
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
  ] 
model = Model(inputs=[input],outputs=layer)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=METRICS)
model.summary()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 16)]              0         
_________________________________________________________________
dense_63 (Dense)             (None, 64)                1088      
_________________________________________________________________
dropout_45 (Dropout)         (None, 64)                0         
_______________________

In [274]:
from tensorflow.keras.callbacks import EarlyStopping
Early=EarlyStopping(patience=100,monitor="val_loss")

In [171]:
import math
def create_learning_rate_scheduler(max_learn_rate=1e-2,
                                   end_learn_rate=1e-6,
                                   warmup_epoch_count=3,
                                   total_epoch_count=15):

    def lr_scheduler(epoch):
        if epoch < warmup_epoch_count:
            res = max_learn_rate
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

    return learning_rate_scheduler

In [313]:
model.fit(TrainDataset,verbose=False,validation_data=ValDataset,epochs=1000,callbacks=[create_learning_rate_scheduler(0.001,0.0005,100,200),Early])




Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00006: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00007: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00008: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00009: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00010: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00011: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00012: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00013: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00014: LearningRateScheduler reducing learning rate to 0.001.

Epoch 00015: LearningRateSchedule

In [276]:
x_train

,jenis_kelamin,umur,hipertensi,penyakit_jantung,sudah_menikah,jenis_tempat_tinggal,rata2_level_glukosa,bmi,merokok
86,0,38,0,0,1,1,0.737502,0.391941,0
241,0,24,0,0,0,1,0.173911,0.183150,0
71,0,49,0,0,1,2,0.028724,0.267399,2
18777,0,64,1,0,1,2,0.228563,0.249084,1
3413,0,56,0,0,1,1,0.045797,0.166056,0
...,...,...,...,...,...,...,...,...,...
6098,0,69,0,1,0,2,0.075241,0.161172,0
17,0,48,0,0,1,1,0.208990,0.332112,0
8586,0,68,1,0,1,1,0.640061,0.202686,0
323,0,45,0,0,1,2,0.158490,0.166056,1


In [314]:
answer = np.argmax(model.predict(test[test.columns[1:]]),axis=1)
test['stroke']=answer
answ=test[["id_pasien","stroke"]]
answ.head()

,id_pasien,stroke
0,2000,0
1,2001,0
2,2002,0
3,2003,0
4,2004,0


In [315]:
answer

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [316]:
test.drop("stroke",axis=1,inplace=True)

In [178]:
answ.to_csv("submit.csv",index=False)

In [327]:
from sklearn.ensemble import RandomForestClassifier

RandomObj = RandomForestClassifier(n_estimators=1000,max_depth=7,random_state=5)
RandomObj.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=5, verbose=0,
                       warm_start=False)

In [328]:
from sklearn.metrics import classification_report
y_pred = RandomObj.predict(x_val)
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.71      0.81        21
         1.0       0.74      0.94      0.83        18

    accuracy                           0.82        39
   macro avg       0.84      0.83      0.82        39
weighted avg       0.85      0.82      0.82        39



In [329]:
Ans =RandomObj.predict(test[test.columns[1:]])

In [331]:
Answer = pd.DataFrame(test["id_pasien"])
Answer["stroke"] = Ans
Answer.to_csv("/content/drive/My Drive/submitmlFixed.csv",index=False)

In [332]:
Answer

,id_pasien,stroke
0,2000,0.0
1,2001,1.0
2,2002,0.0
3,2003,0.0
4,2004,1.0
...,...,...
985,2985,0.0
986,2986,1.0
987,2987,0.0
988,2988,0.0


In [322]:
Ans

array([0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 0.

In [286]:
Ans

array([0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 1., 0.